In [1]:
import yfinance as yf
import time
import math
import pandas as pd
from datetime import date, datetime
from yahoo_fin import options
import numpy as np

import matplotlib.pyplot as plt

from warnings import filterwarnings

filterwarnings("ignore")

In [2]:
symbol = "GME"
RFR = math.e ** (0.0525) - 1

ticker_yahoo = yf.Ticker(symbol)
data = ticker_yahoo.history()

TODAY = data.index[len(data) - 1].date()
print(TODAY)

_price = data["Close"].iloc[-1]
# print(type(_price))

# __price = (data['Close'].iloc[-1]).round(2)
# print(type(__price))

2024-05-06


In [3]:
def _get_iv(chain):
    """get_iv(Object)
    @param Object: chain, an objcet from options.get_options_chain()
    Returns implied volatility as float
    """
    iv = None
    for i in range(len(chain) - 1):
        c = chain["Strike"]
        if c[i + 1] >= _price and c[i] <= _price:
            iv = chain["Implied Volatility"][i]
            break
    return float(iv.strip("%")) / 100

In [4]:
def get_iv(type_options):
    """get_iv(str)
    @param str: type_options, a string to specify either "calls" or "puts"
    Returns tuple of (A) list of expiry dates, and (B) a list of volatilites (iv)
    """
    list_iv = []
    list_expdates = []
    # date_expir_str: str
    for date_expire_str in options.get_expiration_dates(symbol):
        tmp = options.get_options_chain(symbol, date_expire_str[:5])[type_options]
        iv = _get_iv(tmp)
        list_expdates.append(date_expire_str)
        list_iv.append(iv)
        print(f"{date_expire_str}; ATM Call; IV: {iv:.3f}")
    return (list_expdates, list_iv)

In [5]:
list1, list2 = get_iv("calls")

NameError: name 'HTMLSession' is not defined

In [ ]:
# pd.DataFrame({"Expiry": list1, "Implied Volatility": list2}).plot(
#     x="Expiry",
#     y="Implied Volatility",
#     kind="line",
#     xlabel="Expiration Date",
#     ylabel="Implied Volatility",
# )

In [ ]:
# np.average(np.array(list2))

In [ ]:
# list1, list2 = get_iv("puts")

# pd.DataFrame({"Expiry": list1, "Implied Volatility": list2}).plot(
#     x="Expiry",
#     y="Implied Volatility",
#     kind="line",
#     xlabel="Expiration Date",
#     ylabel="Implied Volatility",
# )

In [ ]:
# np.average(np.array(list2))

In [ ]:
# date(2024, 2, 22)


def get_data(con_type: str):
    chains, list_ttm, list_moneyness, list_iv = {}, [], [], []

    for date_expire_str in options.get_expiration_dates(symbol):
        chain = options.get_options_chain(symbol, date_expire_str)[con_type]
        _exp_date = datetime.strptime(
            date_expire_str, "%B %d, %Y"
        ).date()  # .replace(tzinfo=)  # noqa: DTZ007
        ttm = (date(_exp_date.year, _exp_date.month, _exp_date.day) - TODAY).days

        chains[ttm] = chain

        for i in range(len(chain) - 1):
            iv = float(chain["Implied Volatility"][i].strip("%"))
            if iv < 100:
                k = chain["Strike"][i]
                list_ttm.append(ttm)
                list_moneyness.append(_price / k)
                list_iv.append(iv)
    return (chains, list_ttm, list_moneyness, list_iv)

In [ ]:
def do_plots(chains, list_ttm, list_moneyness, list_iv):
    list_x_atm, list_y_atm = [], []

    for ttm in chains.keys():
        _df = chains[ttm]
        df_filtered = _df[(_df["Strike"] > _price)]
        list_x_atm.append(ttm)
        list_y_atm.append(
            float(df_filtered["Implied Volatility"][df_filtered.index[0]].strip("%"))
        )

    plt.plot(list_x_atm, list_y_atm, label="ATM Strike: " + con_type)
    plt.legend()
    plt.xlabel("Time to Maturity (days)")
    plt.ylabel("Implied Volatility (%)")
    plt.show()

    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(projection="3d")
    for i in range(len(list_ttm) - 1):
        ax.scatter(list_ttm[i], list_moneyness[i], list_iv[i], c="b", marker="o")

    ax.set_xlabel("Time to Maturity (Days)")
    ax.set_ylabel("Moneyness")
    ax.set_zlabel("Implied Volatility (%)")
    plt.show()

    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(projection="3d")
    for i in range(len(list_ttm) - 1):
        ax.scatter(list_moneyness[i], list_ttm[i], list_iv[i], marker="o")

    ax.set_xlabel("Moneyness")
    ax.set_ylabel("Time to Maturity (Days)")
    ax.set_zlabel("Implied Volatility (%)")
    plt.show()

In [ ]:
con_type = "calls"
chains_calls, ttm_calls, moneyness_calls, iv_calls = get_data(con_type)
do_plots(chains_calls, ttm_calls, moneyness_calls, iv_calls)

In [ ]:
con_type = "puts"
chains_puts, ttm_puts, moneyness_puts, iv_puts = get_data(con_type)
do_plots(chains_puts, ttm_puts, moneyness_puts, iv_puts)